In [2]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import re
import rhinoMorph
from konlpy.tag import Okt
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [29]:
peter_df = pd.read_csv('./articles_final_peter.csv')
peter_df

,media,date,title,article_original,url
0,조선일보,2020-01-27,방탄소년단 한국 가수 최초 그래미 공연 내년엔 후보 도전,그룹 방탄소년단 BTS 이 26일 현지 시각 한국 가수 최초로 미국 그래미 시싱식에...,https://news.naver.com/main/read.nhn?mode=LSD&...
1,경향신문,2020-02-28,BTS 월드투어 4월 서울 콘서트 취소 코로나19 확산 비상,일부 팬들 환불받아 성금 기탁 트와이스 등 공연 취소 잇따라 코로나19의 확산으...,https://news.naver.com/main/read.nhn?mode=LSD&...
2,조선일보,2020-03-16,방탄소년단 빌보드 200 8위3주 연속 10위권,그룹 방탄소년단 BTS 이 미국 빌보드 메인 앨범 차트인 빌보드 200에서 8위를 ...,https://news.naver.com/main/read.nhn?mode=LSD&...
3,조선일보,2020-04-10,신승훈김건모 위에 BTS 2032만장으로 역대 최다 앨범 판매량 기록,그룹 방탄소년단 BTS 사진 이 한국 가요 역사상 음반을 가장 많이 판 가수가 ...,https://news.naver.com/main/read.nhn?mode=LSD&...
4,조선일보,2020-05-18,이태원 아이돌 BTS 정국 아스트로 차은우 NCT 재현 세븐틴 민규였다,최근 소문으로만 떠돌던 이태원발 코로나 집단 감염 기간 이태원 방문 아이돌이 그룹 ...,https://news.naver.com/main/read.nhn?mode=LSD&...
5,경향신문,2020-06-08,BTS 유튜브 주최 가상 졸업식 디어 클래스 오브 2020 축사 꽃다발은 없어도 가...,경향신문 그룹 방탄소년단 BTS 이 코로나19 확산으로 오프라인 졸업식에 가지 ...,https://news.naver.com/main/read.nhn?mode=LSD&...
6,경향신문,2020-07-23,BTS 75만명 본 방방콘 더 라이브로 기네스 세계기록 추가,경향신문 방탄소년단의 온라인 콘서트 방방콘 더 라이브 이하 방방콘 가 세계에서 ...,https://news.naver.com/main/read.nhn?mode=LSD&...
7,경향신문,2020-08-30,방탄소년단 신곡 영국 오피셜 차트 3위,경향신문 그룹 방탄소년단 BTS사진 의 신곡 다이너마이트 Dynamite 가 세...,https://news.naver.com/main/read.nhn?mode=LSD&...
8,경향신문,2020-09-29,3주 만에 다시 빌보드 정상에 선 방탄소년단,경향신문 그룹 방탄소년단 BTS사진 의 다이너마이트가 3주 만에 미국 빌보드 메...,https://news.naver.com/main/read.nhn?mode=LSD&...
9,경향신문,2020-10-13,병무청 대중문화우수자 병역 연기 입법 추진확정시 BTS 연기 가능,경향신문 병무청이 13일 대중문화예술 분야 우수자의 병역 징집소집 연기 등을 담...,https://news.naver.com/main/read.nhn?mode=LSD&...


In [4]:
jm_df = pd.read_csv('./extData_LDA_ver1.csv')
jm_df

,media,date,title,article_original,url
0,조선일보,2020-01-17,방탄소년단 신보 수록곡 선공개 전 세계 아미 시선 집중,방탄소년단BTS이 정규 앨범 발매에 앞서 수록곡 중 하나를 먼저 선보인다. 지난해 ...,https://news.naver.com/main/read.nhn?mode=LSD&...
1,조선일보,2020-02-22,BTS 새 앨범 전세계 91개 지역 아이튠즈 차트 석권,그룹 방탄소년단BTS의 새 앨범이 발매와 동시에 전 세계 91개 지역에서 아이튠즈 ...,https://news.naver.com/main/read.nhn?mode=LSD&...
2,조선일보,2020-03-16,방탄소년단 빌보드 200 8위3주 연속 10위권,그룹 방탄소년단BTS이 미국 빌보드 메인 앨범 차트인 빌보드 200에서 8위를 차지...,https://news.naver.com/main/read.nhn?mode=LSD&...
3,경향신문,2020-06-08,방탄소년단의 온라인 축사 우리도 서툰 20대우리가 위로와 희망이 되고 싶다,경향신문그룹 방탄소년단BTS이 코로나19 확산으로 오프라인 졸업식에 가지 못하는 졸...,https://news.naver.com/main/read.nhn?mode=LSD&...
4,경향신문,2020-09-15,방탄소년단 미국 빌보드 싱글 차트 2위3주째 최상위권,경향신문 그룹 방탄소년단BTS의 다이너마이트Dynamite가 미국 빌보드 메인 싱글...,https://news.naver.com/main/read.nhn?mode=LSD&...
5,조선일보,2020-09-17,방탄소년단 빅히트 공모주 다이너마이트 터지나,내달56일 코스피 일반 청약 신주 713만주 중 142만주 배정청약증거금 58조원을...,https://news.naver.com/main/read.nhn?mode=LSD&...
6,경향신문,2020-09-24,BTS 필사적으로 자신을 사랑하고 미래를 상상하려 노력하세요,경향신문 2년 만에 유엔 화상 총회 참여 코로나 위기 속 연대의 목소리지금이야말로 ...,https://news.naver.com/main/read.nhn?mode=LSD&...
7,경향신문,2020-10-06,방탄소년단 다이너마이트 6주째 빌보드 싱글 1위 아니면 2위,경향신문그룹 방탄소년단BTS의 다이너마이트가 미국 빌보드 메인 싱글 차트 핫 100...,https://news.naver.com/main/read.nhn?mode=LSD&...
8,중앙일보,2020-10-07,정치권 BTS 병역 특례를 아미 본인들 군대 간다는데 왜,BTS는 빌보드 1위로 1조7000억원의 경제 파급 효과를 냈고 한류 전파와 국위 ...,https://news.naver.com/main/read.nhn?mode=LSD&...
9,중앙일보,2020-10-11,ARXR 첨단기술 총동원1억건 응원 쏟아진 BTS 온라인 콘서트,14만명 75만명 다음은지난해 6월 네이버 브이라이브를 통해 영국 웸블리 공연 생중...,https://news.naver.com/main/read.nhn?mode=LSD&...


In [5]:
will_df = pd.read_csv('./Willver2_rhino10.csv')
will_df

,media,date,title,article_original,url
0,조선일보,2020-01-29,7명의 몸짓을 예술로 현대미술이 BTS라는 물길을 만났다,커넥트 BTS서울 BTS 철학 담은 현대미술 작품 세계 5개도시서 펼치는 프로젝트2...,https://news.naver.com/main/read.nhn?mode=LSD&...
1,한겨레,2020-03-02,BTS 4연속 빌보드 정상비틀스 이래 최단기간 달성 그룹,그룹 방탄소년단BTS의 새 앨범 맵 오브 더 솔7MAP OF THE SOUL7이 영...,https://news.naver.com/main/read.nhn?mode=LSD&...
2,한겨레,2020-06-04,BTS 인종차별과 폭력에 반대우리 모두 존중받을 권리 있어,그룹 방탄소년단BTS이 인종차별과 폭력에 반대한다는 뜻을 밝혔다. 미국에서 백인 경...,https://news.naver.com/main/read.nhn?mode=LSD&...
3,조선일보,2020-09-01,방탄소년단 한국 가수 최초 빌보드 싱글 1위K팝 새역사,2012년 싸이 강남스타일 2위 기록 넘어서방탄소년단BTS이 한국 가수 중 처음으로...,https://news.naver.com/main/read.nhn?mode=LSD&...
4,경향신문,2020-09-03,빅히트 공모주 청약 앞두고 BTS에 주식 증여상장 시 1인당 최대 200억원대,경향신문방탄소년단BTS이 소속사인 빅히트엔터테인먼트로부터 1인당 7만주 가까운 보통...,https://news.naver.com/main/read.nhn?mode=LSD&...
5,조선일보,2020-09-20,다이너마이트 영어로 부른 BTS 방탄 노래는 들려 따라갈 수 있겠다 영상,동영상 뉴스문재인 대통령이 방탄소년단BTS의 노래에 대해 그들의 노래와 춤 모두를 ...,https://news.naver.com/main/read.nhn?mode=LSD&...
6,중앙일보,2020-10-07,아미도 냅두라는데BTS 병역특례 2년만에 다시 띄우는,BTS를 병역특례 해주자는 입장이 아니다. 순수예술인은 특례대상인데 대중예술인은 아...,https://news.naver.com/main/read.nhn?mode=LSD&...
7,중앙일보,2020-10-14,14억 시장에 휘둘린 BTS로이터 중국은 정치적 지뢰,중국 관영 매체 환구시보가 방탄소년단BTS의 수상 소감이 중국 네티즌의 분노를 일으...,https://news.naver.com/main/read.nhn?mode=LSD&...
8,중앙일보,2020-10-20,택배사 또 BTS제품 거부우리가 했겠나 이유 알지 않나,중국 내 5위 규모의 물류업체 윈다가 방탄소년단BTS 관련 제품 배송을 중단하겠다고...,https://news.naver.com/main/read.nhn?mode=LSD&...
9,경향신문,2020-12-01,한글 노래가 빌보드 싱글차트 1위에 방탄소년단 세계 팝시장의 상수가 되다,경향신문방탄소년단BTS이 한국어 곡으로 미국 빌보드 싱글 차트 정상에 올랐다. 빌보...,https://news.naver.com/main/read.nhn?mode=LSD&...


In [6]:
articles_df = pd.concat([peter_df,jm_df,will_df])
articles_df.sort_values(by='date',inplace=True)

In [7]:
articles_df.reset_index(inplace=True)
articles_df.drop('index',inplace=True,axis=1)
articles_df

,media,date,title,article_original,url
0,조선일보,2020-01-17,방탄소년단 신보 수록곡 선공개 전 세계 아미 시선 집중,방탄소년단BTS이 정규 앨범 발매에 앞서 수록곡 중 하나를 먼저 선보인다. 지난해 ...,https://news.naver.com/main/read.nhn?mode=LSD&...
1,조선일보,2020-01-27,방탄소년단 한국 가수 최초 그래미 공연 내년엔 후보 도전,그룹 방탄소년단 BTS 이 26일 현지 시각 한국 가수 최초로 미국 그래미 시싱식에...,https://news.naver.com/main/read.nhn?mode=LSD&...
2,조선일보,2020-01-29,7명의 몸짓을 예술로 현대미술이 BTS라는 물길을 만났다,커넥트 BTS서울 BTS 철학 담은 현대미술 작품 세계 5개도시서 펼치는 프로젝트2...,https://news.naver.com/main/read.nhn?mode=LSD&...
3,조선일보,2020-02-22,BTS 새 앨범 전세계 91개 지역 아이튠즈 차트 석권,그룹 방탄소년단BTS의 새 앨범이 발매와 동시에 전 세계 91개 지역에서 아이튠즈 ...,https://news.naver.com/main/read.nhn?mode=LSD&...
4,경향신문,2020-02-28,BTS 월드투어 4월 서울 콘서트 취소 코로나19 확산 비상,일부 팬들 환불받아 성금 기탁 트와이스 등 공연 취소 잇따라 코로나19의 확산으...,https://news.naver.com/main/read.nhn?mode=LSD&...
5,한겨레,2020-03-02,BTS 4연속 빌보드 정상비틀스 이래 최단기간 달성 그룹,그룹 방탄소년단BTS의 새 앨범 맵 오브 더 솔7MAP OF THE SOUL7이 영...,https://news.naver.com/main/read.nhn?mode=LSD&...
6,조선일보,2020-03-16,방탄소년단 빌보드 200 8위3주 연속 10위권,그룹 방탄소년단BTS이 미국 빌보드 메인 앨범 차트인 빌보드 200에서 8위를 차지...,https://news.naver.com/main/read.nhn?mode=LSD&...
7,조선일보,2020-03-16,방탄소년단 빌보드 200 8위3주 연속 10위권,그룹 방탄소년단 BTS 이 미국 빌보드 메인 앨범 차트인 빌보드 200에서 8위를 ...,https://news.naver.com/main/read.nhn?mode=LSD&...
8,조선일보,2020-04-10,신승훈김건모 위에 BTS 2032만장으로 역대 최다 앨범 판매량 기록,그룹 방탄소년단 BTS 사진 이 한국 가요 역사상 음반을 가장 많이 판 가수가 ...,https://news.naver.com/main/read.nhn?mode=LSD&...
9,조선일보,2020-05-18,이태원 아이돌 BTS 정국 아스트로 차은우 NCT 재현 세븐틴 민규였다,최근 소문으로만 떠돌던 이태원발 코로나 집단 감염 기간 이태원 방문 아이돌이 그룹 ...,https://news.naver.com/main/read.nhn?mode=LSD&...


In [26]:
okt = Okt()

df = okt_morph(articles_df.article_original)
articles_df['morphed'] = df
articles_df

,media,date,title,article_original,url,morphed
0,조선일보,2020-01-17,방탄소년단 신보 수록곡 선공개 전 세계 아미 시선 집중,방탄소년단BTS이 정규 앨범 발매에 앞서 수록곡 중 하나를 먼저 선보인다. 지난해 ...,https://news.naver.com/main/read.nhn?mode=LSD&...,방탄소년단 정규 앨범 발매 앞서 수록 곡 중 먼저 지난해 전 세계 히트 곡 위 시 ...
1,조선일보,2020-01-27,방탄소년단 한국 가수 최초 그래미 공연 내년엔 후보 도전,그룹 방탄소년단 BTS 이 26일 현지 시각 한국 가수 최초로 미국 그래미 시싱식에...,https://news.naver.com/main/read.nhn?mode=LSD&...,그룹 방탄소년단 현지 한국 가수 최초 미국 그래미 싱 공연 방탄소년단 날 미국 로스...
2,조선일보,2020-01-29,7명의 몸짓을 예술로 현대미술이 BTS라는 물길을 만났다,커넥트 BTS서울 BTS 철학 담은 현대미술 작품 세계 5개도시서 펼치는 프로젝트2...,https://news.naver.com/main/read.nhn?mode=LSD&...,커넥트 서울 철학 현대 미술 작품 세계 프로젝트 서울 개막 강 이연 참여 인종 세대...
3,조선일보,2020-02-22,BTS 새 앨범 전세계 91개 지역 아이튠즈 차트 석권,그룹 방탄소년단BTS의 새 앨범이 발매와 동시에 전 세계 91개 지역에서 아이튠즈 ...,https://news.naver.com/main/read.nhn?mode=LSD&...,그룹 방탄소년단 새 앨범 발매 동시 전 세계 개 지역 아이튠즈 차트 위 소속사 빅히...
4,경향신문,2020-02-28,BTS 월드투어 4월 서울 콘서트 취소 코로나19 확산 비상,일부 팬들 환불받아 성금 기탁 트와이스 등 공연 취소 잇따라 코로나19의 확산으...,https://news.naver.com/main/read.nhn?mode=LSD&...,일부 팬 환불 성금 기탁 트와이스 공연 취소 코로나 확산 예정 그룹 방탄소년단 월드...
5,한겨레,2020-03-02,BTS 4연속 빌보드 정상비틀스 이래 최단기간 달성 그룹,그룹 방탄소년단BTS의 새 앨범 맵 오브 더 솔7MAP OF THE SOUL7이 영...,https://news.naver.com/main/read.nhn?mode=LSD&...,그룹 방탄소년단 새 앨범 맵 오브 더 솔 영국 오피 셜 차트 빌보드 팝 차트 위 한...
6,조선일보,2020-03-16,방탄소년단 빌보드 200 8위3주 연속 10위권,그룹 방탄소년단BTS이 미국 빌보드 메인 앨범 차트인 빌보드 200에서 8위를 차지...,https://news.naver.com/main/read.nhn?mode=LSD&...,그룹 방탄소년단 미국 빌보드 메인 앨범 차트 빌보드 위 차지 주 연속 위 빌보드 예...
7,조선일보,2020-03-16,방탄소년단 빌보드 200 8위3주 연속 10위권,그룹 방탄소년단 BTS 이 미국 빌보드 메인 앨범 차트인 빌보드 200에서 8위를 ...,https://news.naver.com/main/read.nhn?mode=LSD&...,그룹 방탄소년단 미국 빌보드 메인 앨범 차트 빌보드 위 차지 주 연속 위 빌보드 현...
8,조선일보,2020-04-10,신승훈김건모 위에 BTS 2032만장으로 역대 최다 앨범 판매량 기록,그룹 방탄소년단 BTS 사진 이 한국 가요 역사상 음반을 가장 많이 판 가수가 ...,https://news.naver.com/main/read.nhn?mode=LSD&...,그룹 방탄소년단 사진 한국 가요 역사상 음반 가장 판 가수 소속사 빅히트 엔터테인먼...
9,조선일보,2020-05-18,이태원 아이돌 BTS 정국 아스트로 차은우 NCT 재현 세븐틴 민규였다,최근 소문으로만 떠돌던 이태원발 코로나 집단 감염 기간 이태원 방문 아이돌이 그룹 ...,https://news.naver.com/main/read.nhn?mode=LSD&...,최근 소문 이태원 발 코로나 집단 감염 기간 이태원 방문 아이돌 그룹 방탄소년단 정...


In [31]:
tfidf = TfidfVectorizer(okt.nouns,lowercase=False)
article_tfidf = tfidf.fit_transform(articles_df.article_original)
article_tfidf

C:\Users\user\anaconda3\lib\site-packages\sklearn\utils\validation.py:67: FutureWarning: Pass input=<bound method Okt.nouns of <konlpy.tag._okt.Okt object at 0x00000222FBFB42E0>> as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "


<36x5230 sparse matrix of type '<class 'numpy.float64'>'
	with 8586 stored elements in Compressed Sparse Row format>

In [76]:
from sklearn.cluster import KMeans

km_cluster = KMeans(n_clusters=10, random_state=42)
km_cluster.fit(article_tfidf)
cluster_label = km_cluster.labels_
cluster_centers = km_cluster.cluster_centers_

In [77]:
articles_df['cluster_label'] = cluster_label
articles_df

,media,date,title,article_original,url,morphed,cluster_label
0,조선일보,2020-01-17,방탄소년단 신보 수록곡 선공개 전 세계 아미 시선 집중,방탄소년단BTS이 정규 앨범 발매에 앞서 수록곡 중 하나를 먼저 선보인다. 지난해 ...,https://news.naver.com/main/read.nhn?mode=LSD&...,방탄소년단 정규 앨범 발매 앞서 수록 곡 중 먼저 지난해 전 세계 히트 곡 위 시 ...,1
1,조선일보,2020-01-27,방탄소년단 한국 가수 최초 그래미 공연 내년엔 후보 도전,그룹 방탄소년단 BTS 이 26일 현지 시각 한국 가수 최초로 미국 그래미 시싱식에...,https://news.naver.com/main/read.nhn?mode=LSD&...,그룹 방탄소년단 현지 한국 가수 최초 미국 그래미 싱 공연 방탄소년단 날 미국 로스...,4
2,조선일보,2020-01-29,7명의 몸짓을 예술로 현대미술이 BTS라는 물길을 만났다,커넥트 BTS서울 BTS 철학 담은 현대미술 작품 세계 5개도시서 펼치는 프로젝트2...,https://news.naver.com/main/read.nhn?mode=LSD&...,커넥트 서울 철학 현대 미술 작품 세계 프로젝트 서울 개막 강 이연 참여 인종 세대...,8
3,조선일보,2020-02-22,BTS 새 앨범 전세계 91개 지역 아이튠즈 차트 석권,그룹 방탄소년단BTS의 새 앨범이 발매와 동시에 전 세계 91개 지역에서 아이튠즈 ...,https://news.naver.com/main/read.nhn?mode=LSD&...,그룹 방탄소년단 새 앨범 발매 동시 전 세계 개 지역 아이튠즈 차트 위 소속사 빅히...,1
4,경향신문,2020-02-28,BTS 월드투어 4월 서울 콘서트 취소 코로나19 확산 비상,일부 팬들 환불받아 성금 기탁 트와이스 등 공연 취소 잇따라 코로나19의 확산으...,https://news.naver.com/main/read.nhn?mode=LSD&...,일부 팬 환불 성금 기탁 트와이스 공연 취소 코로나 확산 예정 그룹 방탄소년단 월드...,8
5,한겨레,2020-03-02,BTS 4연속 빌보드 정상비틀스 이래 최단기간 달성 그룹,그룹 방탄소년단BTS의 새 앨범 맵 오브 더 솔7MAP OF THE SOUL7이 영...,https://news.naver.com/main/read.nhn?mode=LSD&...,그룹 방탄소년단 새 앨범 맵 오브 더 솔 영국 오피 셜 차트 빌보드 팝 차트 위 한...,1
6,조선일보,2020-03-16,방탄소년단 빌보드 200 8위3주 연속 10위권,그룹 방탄소년단BTS이 미국 빌보드 메인 앨범 차트인 빌보드 200에서 8위를 차지...,https://news.naver.com/main/read.nhn?mode=LSD&...,그룹 방탄소년단 미국 빌보드 메인 앨범 차트 빌보드 위 차지 주 연속 위 빌보드 예...,1
7,조선일보,2020-03-16,방탄소년단 빌보드 200 8위3주 연속 10위권,그룹 방탄소년단 BTS 이 미국 빌보드 메인 앨범 차트인 빌보드 200에서 8위를 ...,https://news.naver.com/main/read.nhn?mode=LSD&...,그룹 방탄소년단 미국 빌보드 메인 앨범 차트 빌보드 위 차지 주 연속 위 빌보드 현...,1
8,조선일보,2020-04-10,신승훈김건모 위에 BTS 2032만장으로 역대 최다 앨범 판매량 기록,그룹 방탄소년단 BTS 사진 이 한국 가요 역사상 음반을 가장 많이 판 가수가 ...,https://news.naver.com/main/read.nhn?mode=LSD&...,그룹 방탄소년단 사진 한국 가요 역사상 음반 가장 판 가수 소속사 빅히트 엔터테인먼...,1
9,조선일보,2020-05-18,이태원 아이돌 BTS 정국 아스트로 차은우 NCT 재현 세븐틴 민규였다,최근 소문으로만 떠돌던 이태원발 코로나 집단 감염 기간 이태원 방문 아이돌이 그룹 ...,https://news.naver.com/main/read.nhn?mode=LSD&...,최근 소문 이태원 발 코로나 집단 감염 기간 이태원 방문 아이돌 그룹 방탄소년단 정...,8


In [148]:
top_list = []
for i in range(10):
    # d = km_cluster.transform(article_tfidf)[:,0]
    d = km_cluster.transform(article_tfidf)[:,i]
    # print(d)
    ind = np.argsort(d)[::][:1]
    top_list.append(ind[0])
#     print(ind)
    # articles_df.iloc[ind]
top_list

[21, 7, 23, 19, 31, 35, 11, 26, 25, 18]

In [153]:
articles_df.iloc[top_list].sort_values(by='date')

,media,date,title,article_original,url,morphed,cluster_label
7,조선일보,2020-03-16,방탄소년단 빌보드 200 8위3주 연속 10위권,그룹 방탄소년단 BTS 이 미국 빌보드 메인 앨범 차트인 빌보드 200에서 8위를 ...,https://news.naver.com/main/read.nhn?mode=LSD&...,그룹 방탄소년단 미국 빌보드 메인 앨범 차트 빌보드 위 차지 주 연속 위 빌보드 현...,1
11,경향신문,2020-06-08,BTS 유튜브 주최 가상 졸업식 디어 클래스 오브 2020 축사 꽃다발은 없어도 가...,경향신문 그룹 방탄소년단 BTS 이 코로나19 확산으로 오프라인 졸업식에 가지 ...,https://news.naver.com/main/read.nhn?mode=LSD&...,경향신문 그룹 방탄소년단 코로나 확산 오프라인 졸업식 가지 졸업생 위해 축사 리더 ...,6
18,조선일보,2020-09-17,방탄소년단 빅히트 공모주 다이너마이트 터지나,내달56일 코스피 일반 청약 신주 713만주 중 142만주 배정청약증거금 58조원을...,https://news.naver.com/main/read.nhn?mode=LSD&...,내달 코스피 일반 청약 신주 중 배정 청약 증거 금 코스닥 시장 등판 카카오 게임 ...,9
19,조선일보,2020-09-20,다이너마이트 영어로 부른 BTS 방탄 노래는 들려 따라갈 수 있겠다 영상,동영상 뉴스문재인 대통령이 방탄소년단BTS의 노래에 대해 그들의 노래와 춤 모두를 ...,https://news.naver.com/main/read.nhn?mode=LSD&...,동영상 뉴스 문재인 대통령 방탄소년단 노래 대해 노래 춤 말 강민석 청와대 대변인 ...,3
21,경향신문,2020-09-29,3주 만에 다시 빌보드 정상에 선 방탄소년단,경향신문 그룹 방탄소년단 BTS사진 의 다이너마이트가 3주 만에 미국 빌보드 메...,https://news.naver.com/main/read.nhn?mode=LSD&...,경향신문 그룹 방탄소년단 사진 다이너마이트 주 미국 빌보드 메인 싱글 차트 핫 위 ...,0
23,중앙일보,2020-10-07,정치권 BTS 병역 특례를 아미 본인들 군대 간다는데 왜,BTS는 빌보드 1위로 1조7000억원의 경제 파급 효과를 냈고 한류 전파와 국위 ...,https://news.naver.com/main/read.nhn?mode=LSD&...,빌보드 위로 경제 파급 효과 한류 전파 국위 선양 가치 추정 수 입장 국익 게 더 ...,2
25,중앙일보,2020-10-11,ARXR 첨단기술 총동원1억건 응원 쏟아진 BTS 온라인 콘서트,14만명 75만명 다음은지난해 6월 네이버 브이라이브를 통해 영국 웸블리 공연 생중...,https://news.naver.com/main/read.nhn?mode=LSD&...,지난해 네이버 브이 라이브 통해 영국 웸블리 공연 생중계 휴대폰 앞 지난 위 버스 ...,8
26,중앙일보,2020-10-13,중국엔 정치적 지뢰 있다BTS 지우기에 깜짝 놀란 외신,방탄소년단BTS의 밴플리트상 수상 소감을 두고 중국발 논란이 커지면서 주요 외신들도...,https://news.naver.com/main/read.nhn?mode=LSD&...,방탄소년단 리트 수상 소감 중국 발 논란 주요 외신 주목 일부 중국 네티즌 국가 존...,7
31,경향신문,2020-11-25,방탄소년단 또 새 역사 썼다그래미상 베스트 팝 그룹 퍼포먼스 후보,경향신문 그룹 방탄소년단BTS이 미국 최고 권위 음악시상식인 그래미 어워즈 후보에 ...,https://news.naver.com/main/read.nhn?mode=LSD&...,경향신문 그룹 방탄소년단 미국 최고 권위 음악 시상식 그래미 어워즈 후보 한국 대중...,4
35,경향신문,2020-12-01,한글 노래가 빌보드 싱글차트 1위에 방탄소년단 세계 팝시장의 상수가 되다,경향신문방탄소년단BTS이 한국어 곡으로 미국 빌보드 싱글 차트 정상에 올랐다. 빌보...,https://news.naver.com/main/read.nhn?mode=LSD&...,경향신문 방탄소년단 한국어 곡 미국 빌보드 싱글 차트 정상 빌보드 역사상 최초 기록...,5
